# Evaluating Which Python library is Best suitable for Bulk insert into Aurora Postgres SQL Performance Comparison
###### Soumil Nitin Shah 
###### Hari Om Dubey(Consultant Software Engineer, Python developer)

###### Overview:
* Amazon Aurora PostgreSQL is a fully managed, PostgreSQL–compatible, and ACID–compliant relational database engine that combines the speed, reliability, and manageability of Amazon Aurora with the simplicity and cost-effectiveness of open-source databases. Aurora PostgreSQL is a drop-in replacement for PostgreSQL and makes it simple and cost-effective to set up, operate, and scale your new and existing PostgreSQL deployments, thus freeing you to focus on your business and applications. In this I wanted to test various python libraries for speed and performance and present the findings with community 

##### Step 1 Define Imports 

In [3]:
!pip install psycopg2

     ---------------------------------------- 1.2/1.2 MB 8.2 MB/s eta 0:00:00


In [4]:
!pip install SQLAlchemy

In [7]:
!pip install Faker

     ---------------------------------------- 1.6/1.6 MB 10.5 MB/s eta 0:00:00


In [9]:
!pip install python-dotenv

  Using cached python_dotenv-0.21.0-py3-none-any.whl (18 kB)


In [19]:

try:
    import uuid

    from datetime import datetime
    import os
    import logging

    from functools import wraps
    from enum import Enum
    from abc import ABC, abstractmethod
    from logging import StreamHandler

    import psycopg2
    import os
    import psycopg2.extras as extras
    import sqlalchemy as db
    from dotenv import load_dotenv
    load_dotenv("dev.env")
    from faker import Faker

except Exception as e:
    raise Exception("Error : {}".format(e))

In [20]:
class Logging(object):
    def __init__(self):
        format = "[%(asctime)s] %(name)s %(levelname)s %(message)s"
        # Logs to file
        logging.basicConfig(
            filename="logfile",
            filemode="a",
            format=format,
            level=logging.INFO,
        )
        self.logger = logging.getLogger("python")
        formatter = logging.Formatter(
            format,
        )
        # Logs to Console
        ch = logging.StreamHandler()
        ch.setLevel(logging.INFO)
        ch.setFormatter(formatter)
        self.logger.addHandler(ch)


global logger
logger = Logging()


In [21]:
def error_handling_with_logging(argument=None):
    def real_decorator(function):
        @wraps(function)
        def wrapper(self, *args, **kwargs):
            function_name = function.__name__
            response = None
            try:
                if kwargs == {}:
                    response = function(self)
                else:
                    response = function(self, **kwargs)
            except Exception as e:
                response = {
                    "status": -1,
                    "error": {"message": str(e), "function_name": function.__name__},
                }
                logger.logger.info(response)
            return response

        return wrapper

    return real_decorator


class Settings(object):
    """settings class"""

    def __init__(
            self,
            port="",
            server="",
            username="",
            password="",
            timeout=100,
            database_name="",
            connection_string="",
            collection_name="",
            **kwargs,
    ):
        self.port = port
        self.server = server
        self.username = username
        self.password = password
        self.timeout = timeout
        self.database_name = database_name
        self.connection_string = connection_string
        self.collection_name = collection_name


class DatabaseAurora():
    """Aurora database class"""

    def __init__(self, data_base_settings):
        self.data_base_settings = data_base_settings
        self.client = psycopg2.connect(
            host=self.data_base_settings.server,
            port=self.data_base_settings.port,
            database=self.data_base_settings.database_name,
            user=self.data_base_settings.username,
            password=self.data_base_settings.password,
        )

    @error_handling_with_logging()
    def insert_many(self, query, data):
        self.query = query
        cursor = self.client.cursor()
        cursor.executemany(self.query, data)
        self.client.commit()
        cursor.close()
        return {"statusCode": 200, "data": True}

    @error_handling_with_logging()
    def insert_many_execute_batches(self, query, data):
        self.query = query
        cursor = self.client.cursor()
        extras.execute_batch(cursor, self.query, data)
        self.client.commit()
        cursor.close()
        return {"statusCode": 200, "data": True}


class DatabaseAuroraSqlAlchemy():
    """Aurora database class"""

    def __init__(self, data_base_settings):
        self.data_base_settings = data_base_settings
        self.client = db.create_engine(
            f'postgresql://{self.data_base_settings.username}:{self.data_base_settings.password}@{self.data_base_settings.server}:5432/{self.data_base_settings.database_name}')
        self.metadata = db.MetaData()

    @error_handling_with_logging()
    def insert_many(self, query, data):
        result = self.client.execute(query, data)
        return {"statusCode": 200, "data": True}


class Connector(Enum):
    DB_AURORA_PSYCOPG2 = DatabaseAurora(
        data_base_settings=Settings(
            port=os.getenv("AURORA_DB_PORT"),
            server=os.getenv("AURORA_DB_SERVER"),
            username=os.getenv("AURORA_DB_UID"),
            password=os.getenv("AURORA_DB_PWD"),
            database_name=os.getenv("AURORA_DB_DATABASE"),
        )
    )
    DB_AURORA_SqlAlchemy = DatabaseAuroraSqlAlchemy(
        data_base_settings=Settings(
            port=os.getenv("AURORA_DB_PORT"),
            server=os.getenv("AURORA_DB_SERVER"),
            username=os.getenv("AURORA_DB_UID"),
            password=os.getenv("AURORA_DB_PWD"),
            database_name=os.getenv("AURORA_DB_DATABASE"),
        ))

# Step 2: Generate Data for Test 

In [27]:
global faker
faker = Faker()

class DataGenerator(object):

    @staticmethod
    def get_data():
        name = faker.name().split(" ")
        first_name = name[0]
        last_name = name[1]
        address = faker.address()
        text = faker.text()
        id = uuid.uuid4().__str__()
        city = faker.city()
        state = faker.state()

        _ = {
            "first_name": first_name,
            "last_name": last_name,
            "address": address,
            "text": text,
            "id": id,
            "city": city,
            "state": state
        }
        return _


##### Bulk Insert Data 

In [28]:
batch_data = [tuple(DataGenerator.get_data().values()) for i in range(0, 50000)]

In [30]:
batch_data[0]

('Cindy',
 'Richards',
 '0335 Smith Falls\nWoodardport, NV 24669',
 'Hair imagine miss century nearly car. Be serious yeah beat west. Cut heavy treat may.\nSometimes glass son degree try likely western. Behavior fire economic appear state available mean bring.',
 'deb281af-843a-404d-9615-2670ecd875dd',
 'South Sethport',
 'Kansas')

In [32]:
len(batch_data)

50000

# psycopg2 Execute Batch Method

In [44]:
results = []

helper = Connector.DB_AURORA_PSYCOPG2.value

insert_many_batch_size = [100, 500, 1000, 10000, 30000, 50000]
library_name = "psycopg2"
method = "execute_batches"

for batch in insert_many_batch_size:
    
    query = """INSERT INTO public.user 
                                    ( 
                                    first_name, 
                                    last_name,
                                    address,
                                    text,
                                    id,
                                    city,
                                    state
                                    ) 
                                VALUES (%s, %s, %s, %s, %s, %s,%s)"""
    
    start_time = datetime.now()
    response = helper.insert_many_execute_batches(query=query, data=batch_data[:batch])
    end_time = datetime.now()
    total_time = end_time - start_time

    logger.logger.info(f"""
    ---------------------------
        Report
    ---------------------------

    Library                     : {library_name}
    Batch Size                  : {batch}
    Total Execution Time        : {total_time}
    Total Execution Time Sec    : {total_time.total_seconds()}
    Total Execution Time MS    : {total_time.total_seconds() * 1000}

    """)
        
    results.append(
    {
        "Library":library_name,
        "Batch Size":batch,
        "Total Time in Seconds":total_time.total_seconds(),
        "Total Time in MS":total_time.total_seconds() * 1000,
        "Total Execution Time": total_time
    })

[2022-11-05 12:42:45,874] python INFO 
    ---------------------------
        Report
    ---------------------------

    Library                     : psycopg2
    Batch Size                  : 100
    Total Execution Time        : 0:00:00.105030
    Total Execution Time Sec    : 0.10503
    Total Execution Time MS    : 105.03

    
[2022-11-05 12:42:45,874] python INFO 
    ---------------------------
        Report
    ---------------------------

    Library                     : psycopg2
    Batch Size                  : 100
    Total Execution Time        : 0:00:00.105030
    Total Execution Time Sec    : 0.10503
    Total Execution Time MS    : 105.03

    
[2022-11-05 12:42:46,279] python INFO 
    ---------------------------
        Report
    ---------------------------

    Library                     : psycopg2
    Batch Size                  : 500
    Total Execution Time        : 0:00:00.401870
    Total Execution Time Sec    : 0.40187
    Total Execution Time MS    : 40

In [45]:
results

[{'Library': 'psycopg2',
  'Batch Size': 100,
  'Total Time in Seconds': 0.10503,
  'Total Time in MS': 105.03,
  'Total Execution Time': datetime.timedelta(microseconds=105030)},
 {'Library': 'psycopg2',
  'Batch Size': 500,
  'Total Time in Seconds': 0.40187,
  'Total Time in MS': 401.87,
  'Total Execution Time': datetime.timedelta(microseconds=401870)},
 {'Library': 'psycopg2',
  'Batch Size': 1000,
  'Total Time in Seconds': 0.733407,
  'Total Time in MS': 733.407,
  'Total Execution Time': datetime.timedelta(microseconds=733407)},
 {'Library': 'psycopg2',
  'Batch Size': 10000,
  'Total Time in Seconds': 6.365941,
  'Total Time in MS': 6365.941000000001,
  'Total Execution Time': datetime.timedelta(seconds=6, microseconds=365941)},
 {'Library': 'psycopg2',
  'Batch Size': 30000,
  'Total Time in Seconds': 19.007658,
  'Total Time in MS': 19007.658,
  'Total Execution Time': datetime.timedelta(seconds=19, microseconds=7658)},
 {'Library': 'psycopg2',
  'Batch Size': 50000,
  'Tota

# psycopg2 Execute Many Method

In [46]:
results = []

helper = Connector.DB_AURORA_PSYCOPG2.value

insert_many_batch_size = [100, 500, 1000, 10000, 30000, 50000]
library_name = "psycopg2"
method = "executemany"

for batch in insert_many_batch_size:
    
    query = """INSERT INTO public.user 
                                    ( 
                                    first_name, 
                                    last_name,
                                    address,
                                    text,
                                    id,
                                    city,
                                    state
                                    ) 
                                VALUES (%s, %s, %s, %s, %s, %s,%s)"""
    
    start_time = datetime.now()
    response = helper.insert_many(query=query, data=batch_data[:batch])
    end_time = datetime.now()
    total_time = end_time - start_time

    logger.logger.info(f"""
    ---------------------------
        Report
    ---------------------------

    Library                     : {library_name}
    Batch Size                  : {batch}
    Total Execution Time        : {total_time}
    Total Execution Time Sec    : {total_time.total_seconds()}
    Total Execution Time MS    : {total_time.total_seconds() * 1000}

    """)
        
    results.append(
    {
        "Library":library_name,
        "Batch Size":batch,
        "Total Time in Seconds":total_time.total_seconds(),
        "Total Time in MS":total_time.total_seconds() * 1000,
        "Total Execution Time": total_time
    })

[2022-11-05 12:43:55,446] python INFO 
    ---------------------------
        Report
    ---------------------------

    Library                     : psycopg2
    Batch Size                  : 100
    Total Execution Time        : 0:00:04.433265
    Total Execution Time Sec    : 4.433265
    Total Execution Time MS    : 4433.264999999999

    
[2022-11-05 12:43:55,446] python INFO 
    ---------------------------
        Report
    ---------------------------

    Library                     : psycopg2
    Batch Size                  : 100
    Total Execution Time        : 0:00:04.433265
    Total Execution Time Sec    : 4.433265
    Total Execution Time MS    : 4433.264999999999

    
[2022-11-05 12:44:17,028] python INFO 
    ---------------------------
        Report
    ---------------------------

    Library                     : psycopg2
    Batch Size                  : 500
    Total Execution Time        : 0:00:21.581738
    Total Execution Time Sec    : 21.581738
    Total

In [47]:
results

[{'Library': 'psycopg2',
  'Batch Size': 100,
  'Total Time in Seconds': 4.433265,
  'Total Time in MS': 4433.264999999999,
  'Total Execution Time': datetime.timedelta(seconds=4, microseconds=433265)},
 {'Library': 'psycopg2',
  'Batch Size': 500,
  'Total Time in Seconds': 21.581738,
  'Total Time in MS': 21581.738,
  'Total Execution Time': datetime.timedelta(seconds=21, microseconds=581738)},
 {'Library': 'psycopg2',
  'Batch Size': 1000,
  'Total Time in Seconds': 43.32151,
  'Total Time in MS': 43321.51,
  'Total Execution Time': datetime.timedelta(seconds=43, microseconds=321510)},
 {'Library': 'psycopg2',
  'Batch Size': 10000,
  'Total Time in Seconds': 433.394085,
  'Total Time in MS': 433394.085,
  'Total Execution Time': datetime.timedelta(seconds=433, microseconds=394085)},
 {'Library': 'psycopg2',
  'Batch Size': 30000,
  'Total Time in Seconds': 1305.804741,
  'Total Time in MS': 1305804.741,
  'Total Execution Time': datetime.timedelta(seconds=1305, microseconds=804741)

# sqlalchemy Inserts Batch

In [48]:
results = []

helper = Connector.DB_AURORA_SqlAlchemy.value

insert_many_batch_size = [100, 500, 1000, 10000, 30000, 50000]
library_name = "sqlalchemy"
method = "execute"

for batch in insert_many_batch_size:
    
    schema = db.Table(
            'user',
            db.MetaData(),
            db.Column('first_name', db.String(255), nullable=False),
            db.Column('last_name', db.String(255), nullable=False),
            db.Column('address', db.String(255), nullable=False),
            db.Column('text', db.String(255), nullable=False),
            db.Column('id', db.String(255), nullable=False),
            db.Column('city', db.String(255), nullable=False),
            db.Column('state', db.String(255), nullable=False)
        )
    
    query = db.insert(schema)
    
    start_time = datetime.now()
    response = helper.insert_many(query=query, data=batch_data[:batch])
    end_time = datetime.now()
    
    total_time = end_time - start_time

    logger.logger.info(f"""
    ---------------------------
        Report
    ---------------------------

    Library                     : {library_name}
    Batch Size                  : {batch}
    Total Execution Time        : {total_time}
    Total Execution Time Sec    : {total_time.total_seconds()}
    Total Execution Time MS    : {total_time.total_seconds() * 1000}

    """)
        
    results.append(
    {
        "Library":library_name,
        "Batch Size":batch,
        "Total Time in Seconds":total_time.total_seconds(),
        "Total Time in MS":total_time.total_seconds() * 1000,
        "Total Execution Time": total_time
    })

[2022-11-05 14:14:47,240] python INFO 
    ---------------------------
        Report
    ---------------------------

    Library                     : sqlalchemy
    Batch Size                  : 100
    Total Execution Time        : 0:00:00.855068
    Total Execution Time Sec    : 0.855068
    Total Execution Time MS    : 855.0680000000001

    
[2022-11-05 14:14:47,240] python INFO 
    ---------------------------
        Report
    ---------------------------

    Library                     : sqlalchemy
    Batch Size                  : 100
    Total Execution Time        : 0:00:00.855068
    Total Execution Time Sec    : 0.855068
    Total Execution Time MS    : 855.0680000000001

    
[2022-11-05 14:14:47,389] python INFO 
    ---------------------------
        Report
    ---------------------------

    Library                     : sqlalchemy
    Batch Size                  : 500
    Total Execution Time        : 0:00:00.149842
    Total Execution Time Sec    : 0.149842
    

In [49]:
results

[{'Library': 'sqlalchemy',
  'Batch Size': 100,
  'Total Time in Seconds': 0.855068,
  'Total Time in MS': 855.0680000000001,
  'Total Execution Time': datetime.timedelta(microseconds=855068)},
 {'Library': 'sqlalchemy',
  'Batch Size': 500,
  'Total Time in Seconds': 0.149842,
  'Total Time in MS': 149.842,
  'Total Execution Time': datetime.timedelta(microseconds=149842)},
 {'Library': 'sqlalchemy',
  'Batch Size': 1000,
  'Total Time in Seconds': 0.135259,
  'Total Time in MS': 135.259,
  'Total Execution Time': datetime.timedelta(microseconds=135259)},
 {'Library': 'sqlalchemy',
  'Batch Size': 10000,
  'Total Time in Seconds': 0.632779,
  'Total Time in MS': 632.779,
  'Total Execution Time': datetime.timedelta(microseconds=632779)},
 {'Library': 'sqlalchemy',
  'Batch Size': 30000,
  'Total Time in Seconds': 1.722918,
  'Total Time in MS': 1722.918,
  'Total Execution Time': datetime.timedelta(seconds=1, microseconds=722918)},
 {'Library': 'sqlalchemy',
  'Batch Size': 50000,
  '